<a href="https://colab.research.google.com/github/anandita-amalia/HTGAA-2025/blob/main/colab_notebook_for_ml_models_Anandita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Libraries

In [ ]:
!pip install aiohttp

# Set password for this notebook

In [ ]:
auth_token = "hgtaaproteindesign2025" # "as-C8Ys4f9HASQilsKbtJPYPK"
headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
}

# Cleaning and Renumbering your PDB

Sometimes, PDBs will start at position 20, this is because PDB's often contain sub components of a bigger protein sequence (You can refer to [UniProt](https://www.uniprot.org/) for the full sequence). If you want to renumber your protein to be numbered from position 1, use the following code. This will also clean your protein by fixing any chain breaks etc.

In [ ]:
import aiohttp
import asyncio
from google.colab import files

# @markdown MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTL
# @markdown VTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLV
# @markdown NRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLAD
# @markdown HYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK
upload_dict = files.upload()
pdb_string = upload_dict[list(upload_dict.keys())[0]]
with open("tmp.pdb","wb") as out: out.write(pdb_string)


async def post_clean_pdb(url: str, pdb_string: str):
    payload = {
        "pdb_string": pdb_string
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload,timeout=60*60,headers=headers) as response:
            response_json = await response.json()
            return response_json

url = "https://molecularmachines--pdbfixer-fastapi-app.modal.run/fixpdb"
response_data = await post_clean_pdb(url, open("tmp.pdb","r").read())
filename = list(upload_dict.keys())[0]
filename_fixed= f"{filename.replace('.pdb','')}_fixed.pdb"
with open(filename_fixed,"w") as f:
  f.write(response_data["data"])
import aiohttp
import asyncio
from google.colab import files

#@markdown #### Upload a PDB File
upload_dict = files.upload()
pdb_string = upload_dict[list(upload_dict.keys())[0]]
with open("tmp.pdb","wb") as out: out.write(pdb_string)






In [ ]:
pdb_string

# ProteinMPNN : Generating Sequences From a Protein Structure

This code runs the ProteinMPNN model and gives you a set of sequences for the pdb sturcture provided. If your pdb has more than chain, then you should also specify the chain that you want to make proposal sequences for. Note that these sequences are predicted to fold into the same structure as the provided PDB but there is no guarantee that they will perform the same function.  

In [ ]:
import aiohttp
import asyncio
from google.colab import files

#@markdown #### Upload a PDB File
chains = "A" #@param {type:"string"}
temp = "0.1" #@param {type:"string"}
num_seqs = "1" #@param {type:"string"}
#@markdown #### Fix some positions in the protein sequence and only sample new amino acids at other positions.
fix_pos = "1" #@param {type:"string"}
#@markdown - specify which positions to keep fixed in the sequence (example: `1,2-10`)
#@markdown - you can also specify chain specific constraints (example: `A1-10,B1-20`)
#@markdown - you can also specify to fix entire chain(s) (example: `A`)

#@markdown #### If you want to do the inverse of the fix positions where you only sample at the residues specified but not at the other locations check the following box
inverse = True #@param {type:"boolean"}


upload_dict = files.upload()
pdb_string = upload_dict[list(upload_dict.keys())[0]]
with open("tmp.pdb","wb") as out: out.write(pdb_string)

auth_token = "hgtaaproteindesign2025"  # Define auth_token here
headers = {
    "Authorization": f"Bearer {auth_token}",
    "Content-Type": "application/json"
}  # Define headers here

async def post_sample_request(url: str, pdb_string: str, mpnn_config: dict):
    payload = {
        "pdb_string": pdb_string,
        "params": mpnn_config
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload,timeout=60*60,headers=headers) as response:
            try:
                text_response = await response.text()
                print("Raw response:", text_response)
                response_json = await response.json()
                return response_json
            except Exception as e:
                print("Error processing response:", e)
                print("Response content:", await response.text())
                return {"error": str(e)}

url = "https://molecularmachines--colabdesign-fastapi-app.modal.run/sample"
pdb_string = open("tmp.pdb","r").read()

mpnn_config = {
    "fix_pos": fix_pos,
    "inverse": inverse,
    "temp": float(temp),
    "batch": int(num_seqs),
    "chains": chains
}
response_data = await post_sample_request(url, pdb_string, mpnn_config)
response_data

# AlphaFold : Fold a protein using it's sequence

This code will run alphafold model and return a PDB structure that is the predicted structure for that sequence. You can evaluate how good the predicted structure is by using how confident AlphaFold is about its prediction. High confidence is usually a good indicator for correlating with the ground truth structure.

In [ ]:
import aiohttp
import asyncio
from google.colab import files


#@markdown #### Input the sequence you would like to fold. Output will be in the files under ```output_predicted.pdb```
sequence = "MASSVTPLGSWVLLHHHPSTILTQSRSRSPPSLITLKPISLTPKRTVSSSSSSSLITKEDNNLKSSSSSFDFMSYIIRKADSVNKALDSAVPLREPLKIHEAMRYSLLAGGKRVRPVLCIAACELVGGEESLAMPARCAVEMIHTMSLIHDDLPCMDNDDLRRGKPTNHKVYGEDVAVLA GDALLSFAFEHLASATSSEVSPARVVRAVGELAKAIGTEGLVAGQVVDISSEGLDLNNVGLEHLKFIHLHKTAALLEASAVLGGIIGGGSDEEIERLRKFARCIGLLFQVVDDILDVTKSSQELGKTAGKDLIADKLTYPKLMGLEKSREFAEKLNTEARDQLLGFDSDKVAPLLALANYIANRQN" #@param {type:"string"}

auth_token = "hgtaaproteindesign2025"  # Replace with your actual auth token
headers = {
    "Authorization": f"Bearer {auth_token}",
    "Content-Type": "application/json"
}
async def post_fold_request(url:str,sequence:str):
    payload = {
        "sequence": sequence
    }
    print("starting to communicate")
    async with aiohttp.ClientSession() as session:
        print("session created")
        async with session.post(url, json=payload,headers=headers,timeout=60*60) as response:
            response_json = await response.json()
            print("complete, json found and returning")
            return response_jsonimport

alphafold_url = "https://molecularmachines--colabfold-fastapi-app.modal.run/alphafold/monomer"

response_data = await post_fold_request(alphafold_url, sequence)
with open("output_predicted.pdb","w") as f:
  f.write(response_data['results']['predicted_output'])

# AlphaFold-Multimer Fold multiple sequences to predict protein complex structure

This code will run alphafold model and return a PDB structure that is the predicted structure of two sequences. You can evaluate how good the predicted structure is by using how confident AlphaFold is about its prediction. High confidence is usually a good indicator for correlating with the ground truth structure.

In [ ]:
import aiohttp
import asyncio

#@markdown #### Input the sequences you would like to fold. Output will be in the files under ```output_predicted_complex.pdb```
sequence1 = "MADQLTDEQIAEFKEAFSLFDKDGDGCITTKELGTVMRSLGQNPTEAELQDMINEVDADG NGTIDFPEFLNLMARKMKDTDSEEELKEAFRVFDKDQDGFISAAELRHVMTNLGEKLTDE EVDEMIREADVDGDGQINYEEFVKVMMAK" #@param {type:"string"}
# sequence2 = "MGVHECPAWLWLLLSLLSLPLGLPVLGAPPRLICDSRVLERYLLEAKEAENITTGCAEHCSLNENITVPDTKVNFYAWKRMEVGQQAVEVWQGLALLSEAVLRGQALLVNSSQPWEPLQLHVDKAVSGLRSLTTLLRALGAQKEAISPPDAASAAPLRTITADTFRKLFRVYSNFLRGKLKLYTGEACRTGDR" #@param {type:"string"}

async def post_multimer_request(url:str,sequence1:str,sequence2:str):
    payload = {
        "sequences": [sequence1],
        "target_sequence":sequence2
    }
    async with aiohttp.ClientSession() as session:
      async with session.post(url, json=payload,headers=headers,timeout=60*60) as response:
            response_json = await response.json()
            return response_json
url = "https://molecularmachines--colabfold-fastapi-app.modal.run/alphafold/multimer"

response_data = await post_multimer_request(url, sequence1, sequence2)
pdb_string = response_data["results"][0]["pdb_str"]
with open("output_predicted_complex.pdb","w") as f:
  f.write(pdb_string)

In [ ]:
import pymol
from pymol import cmd
import requests

# Function to predict structure using ESMFold API (faster than AlphaFold)
def predict_structure_fasta(sequence, output_file="predicted_gfp.pdb"):
    url = "https://api.esmatlas.com/foldSequence/v1/pdb/"
    response = requests.post(url, data=sequence)
    if response.status_code == 200:
        with open(output_file, "w") as f:
            f.write(response.text)
        print("Predicted structure saved as", output_file)
    else:
        print("Error predicting structure:", response.text)

# GFP amino acid sequence (from UniProt P42212)
gfp_sequence = """MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"""

# Predict structure and save as PDB
predict_structure_fasta(gfp_sequence)

# Start PyMOL
pymol.finish_launching()

# Load the experimental GFP structure from RCSB PDB
cmd.fetch('1GFL', 'real_gfp')

# Load the predicted structure
cmd.load('predicted_gfp.pdb', 'pred_gfp')

# Align predicted and real structure
cmd.align('pred_gfp', 'real_gfp')

# Color the structures for visualization
cmd.color('cyan', 'real_gfp')  # Experimental structure in cyan
cmd.color('magenta', 'pred_gfp')  # Predicted structure in magenta

# Show as cartoon representation
cmd.show('cartoon', 'real_gfp')
cmd.show('cartoon', 'pred_gfp')
cmd.png('gfp_real_vs_predicted.png')
cmd.refresh()

print("Comparison complete. Check the PyMOL visualization.")


# Embed a protein sequence with ESMC model

Embeds a protein sequence with ESMC - https://www.evolutionaryscale.ai/blog/esm-cambrian and gives you a list of numbers which is the embedding of that sequence

In [ ]:
import aiohttp
import asyncio

#@markdown #### Input the sequence you would like to fold. Output will be in the files under ```output_predicted.pdb```
sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTL VTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLV NRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLAD HYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK" #@param {type:"string"}

async def post_embedding(url:str,sequence:str):
    payload = {
        "sequence": sequence
    }

 async with aiohttp.ClientSession() as session:
    async with session.post(url, json=payload,headers=headers) as response:
            # Check if the response is successful before attempting to parse JSON
            if response.status == 200:
                response_json = await response.json()
                return response_json
            else:
                # Handle the error, e.g., print the error message or raise an exception
                error_text = await response.text()
                print(f"Error: {response.status} - {error_text}")
                # You can choose to re-raise the exception or return an error value
                # raise Exception(f"Error: {response.status} - {error_text}")
                return {"error": error_text}  # Or return an error dictionary

esm_url = "https://molecularmachines--esm-fast-app.modal.run/esm/embedding"

response_data = await post_embedding(esm_url, sequence)
embedding = response_data

# Check if there was an error before accessing the 'pll' key
if "error" not in embedding:
    print(f"pseudo log likelihood score is {embedding['pll']}")
else:
    print("Could not retrieve embedding. See error message above.")

# Make a binder with BindCraft

Make a binder for the provided target pdb

In [ ]:
import aiohttp
import asyncio
from google.colab import files
#@markdown #### Input the sequence you would like to fold. Output will be in the files under ```output_predicted.pdb```
upload_dict = files.upload()
pdb_string = upload_dict[list(upload_dict.keys())[0]]
with open("tmp.pdb","wb") as out: out.write(pdb_string)


chains = "A" #@param {type:"string"}
binder_length_range = "3,4" #@param {type:"string"}
num_seqs = "1" #@param {type:"string"}
target_hotspot_residues = "A1-100" # @param {"type":"string","placeholder":""}

async def post_embedding(url:str):
    payload={
                "pdb_str": open("tmp.pdb","r").read(),
                "chains": chains,
                "lengths": binder_length_range,
                "num_samples": int(num_seqs),
                "fold_conditioning": "Default",
                "target_hotspot_residues":target_hotspot_residues
            }

    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload,headers=headers,timeout=60*60) as response:
            response_json = await response.json()
            return response_json

esm_url = "https://molecularmachines--bindcraft-fast-app.modal.run/bindcraft/design_binder_wrapper"

response_data = await post_embedding(esm_url)
response_data

# Fold a protein and ligand with boltz-1



In [ ]:
import aiohttp
import asyncio
from google.colab import files
import json
import aiohttp


protein_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTL VTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLV NRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLAD HYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK" #@param {type:"string"}
ligand_sequence = "N[C@@H](Cc1ccc(O)cc1)C(=O)O" #@param {type:"string"}
async def post_embedding(url:str):
    payload={
              "protein_sequence":protein_sequence,
              "ligand_sequence":ligand_sequence
            }

    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload,headers=headers,timeout=60*60) as response:
            output_bytes = await response.read()

            # Write the received bytes to a file named 'output.zip'
            with open("boltz_output.zip", "wb") as f:
                f.write(output_bytes)

            return "Response written to boltz_output.zip"

esm_url = "https://molecularmachines--boltz1-fastapi-app.modal.run/infer"

response_data = await post_embedding(esm_url)
response_data

In [ ]:
from google.colab import files
files.download("boltz_output.zip")

In [ ]:
import aiohttp
import asyncio
from google.colab import files
import requests
import pymol
from pymol import cmd

# Install necessary libraries
!pip install aiohttp

# Set authentication token
auth_token = "hgtaaproteindesign2025"
headers = {
    "Authorization": f"Bearer {auth_token}",
    "Content-Type": "application/json"
}

# GFP amino acid sequence
gfp_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

# Predict structure using AlphaFold API
async def post_fold_request(url: str, sequence: str):
    payload = {"sequence": sequence}
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload, headers=headers, timeout=60*60) as response:
            response_json = await response.json()
            return response_json

# AlphaFold URL
alphafold_url = "https://molecularmachines--colabfold-fastapi-app.modal.run/alphafold/monomer"

# Predict GFP structure
response_data = await post_fold_request(alphafold_url, gfp_sequence)

# Save predicted structure
with open("predicted_gfp.pdb", "w") as f:
    f.write(response_data['results']['predicted_output'])

# Load real and predicted structures into PyMOL
pymol.finish_launching()
cmd.fetch('1GFL', 'real_gfp')
cmd.load('predicted_gfp.pdb', 'pred_gfp')
cmd.align('pred_gfp', 'real_gfp')

# Color and visualize
cmd.color('cyan', 'real_gfp')  # Experimental structure in cyan
cmd.color('magenta', 'pred_gfp')  # Predicted structure in magenta
cmd.show('cartoon', 'real_gfp')
cmd.show('cartoon', 'pred_gfp')
cmd.png('gfp_real_vs_predicted.png')
cmd.refresh()

print("Comparison complete. Check the PyMOL visualization.")


In [ ]:
import aiohttp
import asyncio
from google.colab import files
import requests
import pymol
from pymol import cmd

# Install necessary libraries
!pip install aiohttp

# Set authentication token
auth_token = "hgtaaproteindesign2025"
headers = {
    "Authorization": f"Bearer {auth_token}",
    "Content-Type": "application/json"
}

# GFP amino acid sequence
gfp_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

# Predict structure using AlphaFold API
async def post_fold_request(url: str, sequence: str):
    payload = {"sequence": sequence}
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload, headers=headers, timeout=60*60) as response:
            response_json = await response.json()
            return response_json

# AlphaFold URL
alphafold_url = "https://molecularmachines--colabfold-fastapi-app.modal.run/alphafold/monomer"

# Predict GFP structure
response_data = await post_fold_request(alphafold_url, gfp_sequence)

# Save predicted structure
with open("predicted_gfp.pdb", "w") as f:
    f.write(response_data['results']['predicted_output'])

# Load real and predicted structures into PyMOL
pymol.finish_launching()
cmd.fetch('1GFL', 'real_gfp')
cmd.load('predicted_gfp.pdb', 'pred_gfp')
cmd.align('pred_gfp', 'real_gfp')

# Color and visualize
cmd.color('cyan', 'real_gfp')  # Experimental structure in cyan
cmd.color('magenta', 'pred_gfp')  # Predicted structure in magenta
cmd.show('cartoon', 'real_gfp')
cmd.show('cartoon', 'pred_gfp')
cmd.png('gfp_real_vs_predicted.png')
cmd.refresh()

print("Comparison complete. Check the PyMOL visualization.")


In [ ]:
# Install required packages
!pip install git+https://github.com/sokrypton/ColabDesign.git
!pip install git+https://github.com/sokrypton/ColabFold.git
!pip install torch torchvision torchaudio

# Clone ProteinMPNN repository
!git clone https://github.com/sokrypton/ColabDesign.git
!git clone https://github.com/sokrypton/ProteinMPNN.git
%cd ProteinMPNN
!pip install -e .

# Import necessary libraries
import torch
import numpy as np
from colabdesign.mpnn import mk_mpnn_model
from colabdesign.shared.protein import parse_pdb
import os

# Load GFP protein structure (PDB format)
gfp_pdb = """\
ATOM      1  N   MET A   1      20.154  30.399   5.673  1.00 32.20           N
ATOM      2  CA  MET A   1      21.590  30.174   5.861  1.00 28.68           C
ATOM      3  C   MET A   1      22.269  29.963   4.513  1.00 29.23           C
... (trimmed for brevity, use full GFP PDB file here)
"""

# Save the GFP PDB file
pdb_filename = "gfp.pdb"
with open(pdb_filename, "w") as f:
    f.write(gfp_pdb)

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mpnn_model = mk_mpnn_model()
mpnn_model.to(device)

# Parse the PDB structure
protein_structure = parse_pdb(pdb_filename)

# Perform reverse folding
num_seqs = 5  # Number of sequences to generate
mpnn_seqs = mpnn_model.design(protein_structure, num=num_seqs)

# Print the generated sequences
for i, seq in enumerate(mpnn_seqs):
    print(f"Generated Sequence {i+1}: {seq}")


In [ ]:
# Install required packages
!pip install git+https://github.com/sokrypton/ColabDesign.git
!pip install git+https://github.com/sokrypton/ColabFold.git
!pip install torch torchvision torchaudio biopython py3Dmol

# Clone ProteinMPNN repository
!git clone https://github.com/sokrypton/ColabDesign.git
!git clone https://github.com/sokrypton/ProteinMPNN.git
%cd ProteinMPNN
!pip install -e .

# Import necessary libraries
import torch
import numpy as np
import py3Dmol
from colabdesign.mpnn import mk_mpnn_model
from colabdesign.shared.protein import parse_pdb
from Bio import PDB
import os

# Load GFP protein structure (PDB format)
gfp_pdb = """\
ATOM      1  N   MET A   1      20.154  30.399   5.673  1.00 32.20           N
ATOM      2  CA  MET A   1      21.590  30.174   5.861  1.00 28.68           C
ATOM      3  C   MET A   1      22.269  29.963   4.513  1.00 29.23           C
... (trimmed for brevity, use full GFP PDB file here)
"""

# Save the GFP PDB file
pdb_filename = "gfp.pdb"
with open(pdb_filename, "w") as f:
    f.write(gfp_pdb)

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mpnn_model = mk_mpnn_model()
mpnn_model.to(device)

# Parse the PDB structure
protein_structure = parse_pdb(pdb_filename)

# Perform reverse folding to generate sequences
num_seqs = 5  # Number of sequences to generate
mpnn_seqs = mpnn_model.design(protein_structure, num=num_seqs)

# Save the first generated sequence as a FASTA file
seq_filename = "reverse_folded_gfp.fasta"
with open(seq_filename, "w") as f:
    f.write(">Reverse_Folded_GFP\n" + mpnn_seqs[0])

# Print the generated sequence
print(f"Generated Reverse-Folded Sequence:\n{mpnn_seqs[0]}")

# Visualizing the folded structure using py3Dmol
def show_pdb(pdb_filename):
    with open(pdb_filename, "r") as file:
        pdb_content = file.read()

    view = py3Dmol.view(width=600, height=600)
    view.addModel(pdb_content, "pdb")
    view.setStyle({"cartoon": {"color": "spectrum"}})
    view.zoomTo()
    return view.show()

# Display the 3D folded structure
show_pdb(pdb_filename)


In [ ]:
# Install required packages
!pip install git+https://github.com/sokrypton/ColabDesign.git
!pip install git+https://github.com/sokrypton/ColabFold.git
!pip install torch torchvision torchaudio biopython py3Dmol

# Clone ProteinMPNN repository and install
!git clone https://github.com/sokrypton/ColabDesign.git
!git clone https://github.com/sokrypton/ProteinMPNN.git
%cd ProteinMPNN
!pip install -e .

# Import necessary libraries
import torch
import numpy as np
import py3Dmol
from colabdesign.af.alphafold.common import protein # Import protein module from the correct location
from Bio import PDB
import os


# Load GFP protein structure (PDB format)
gfp_pdb = """\
ATOM      1  N   MET A   1      20.154  30.399   5.673  1.00 32.20           N
ATOM      2  CA  MET A   1      21.590  30.174   5.861  1.00 28.68           C
ATOM      3  C   MET A   1      22.269  29.963   4.513  1.00 29.23           C
... (trimmed for brevity, use full GFP PDB file here)
"""

# Save the GFP PDB file
pdb_filename = "gfp.pdb"
with open(pdb_filename, "w") as f:
    f.write(gfp_pdb)

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Import mk_mpnn_model directly from colabdesign.mpnn
from colabdesign.mpnn import mk_mpnn_model
mpnn_model = mk_mpnn_model()
mpnn_model.to(device)

# Parse the PDB structure using protein.from_pdb_string
protein_structure = protein.from_pdb_string(gfp_pdb)

# Perform reverse folding to generate sequences
num_seqs = 5  # Number of sequences to generate
mpnn_seqs = mpnn_model.design(protein_structure, num=num_seqs)

# Save the first generated sequence as a FASTA file
seq_filename = "reverse_folded_gfp.fasta"
with open(seq_filename, "w") as f:
    f.write(">Reverse_Folded_GFP\n" + mpnn_seqs[0])

# Print the generated sequence
print(f"Generated Reverse-Folded Sequence:\n{mpnn_seqs[0]}")

# Visualizing the folded structure using py3Dmol
def show_pdb(pdb_filename):
    with open(pdb_filename, "r") as file:
        pdb_content = file.read()

    view = py3Dmol.view(width=600, height=600)
    view.addModel(pdb_content, "pdb")
    view.setStyle({"cartoon": {"color": "spectrum"}})
    view.zoomTo()
    return view.show()

# Display the 3D folded structure
show_pdb(pdb_filename)